In [2]:
import os
import re
import csv
import glob
import time
from collections import OrderedDict
import pandas as pd

## read Category:Princess (プリンセス）articles data downloaded from PetScan (https://petscan.wmflabs.org/?psid=22392988)
### User manual (https://meta.wikimedia.org/wiki/PetScan/)
-  read download.csv (プリンセス|2　生年|20　人物|20)

In [3]:
df = pd.read_csv("princess.csv")
df.head(20)

,number,title,pageid,namespace,length,touched
0,1,メアリー1世_(イングランド女王),26741,NaN,25534,20220520220448
1,2,ヴィクトリア_(ドイツ皇后),81754,NaN,20973,20220511064128
2,3,アリス_(ヘッセン大公妃),90897,NaN,8258,20220530161026
3,4,ベアトリス_(イギリス王女),91600,NaN,6248,20210307075642
4,5,ダイアナ_(プリンセス・オブ・ウェールズ),91976,NaN,141459,20220614100009
5,6,キャサリン・オブ・アラゴン,105591,NaN,58037,20220607152830
6,7,マデレーン_(スウェーデン王女),152555,NaN,10895,20220607100652
7,8,ヴィクトリア_(スウェーデン皇太子),152848,NaN,15646,20220607100745
8,9,アンナ・パヴロヴナ,166869,NaN,5436,20210821235655
9,10,レーニエ3世,187272,NaN,8520,20220420092855


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 428 entries, 0 to 427
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   number     428 non-null    int64  
 1   title      428 non-null    object 
 2   pageid     428 non-null    int64  
 3   namespace  0 non-null      float64
 4   length     428 non-null    int64  
 5   touched    428 non-null    int64  
dtypes: float64(1), int64(4), object(1)
memory usage: 20.2+ KB


In [5]:
# key: folder's name, value: create dictionartris of max IDs
# exrtract max ID values from the last article in the last file in each directory
def mapping_dir(dir_path):
    dir_list = os.listdir(dir_path)
    for _dir in dir_list:
        last_f_path = sorted(glob.glob(os.path.join(dir_path, _dir) + '\\*'))[-1]
        max_id, _, _ = format_text(sorted(get_articlelist(last_f_path))[-1])
        dir_info[_dir] = max_id
    return dir_info

# pade_id from csv file and dir_info created by mapping_dir
# return folder name which include page_id I'm looking for
def get_search_dir(page_id, dir_info):
    min_id = 0
    for dir_name, max_id in dir_info.items():
        search_dir = dir_name
        if min_id <= page_id <= max_id:
            return search_dir
        else:
            min_id = max_id
    return next(iter(dir_info))     # if not found, retuen the first value(AA)
 
# If nothing is found with search_dir, search again from the beginning
# this can be replaced by "get_files"
def get_all_files(base_path, search_dir):
    dir_iter = iter(dir_info.keys())   # in case search_dir fails
    dir_list = list(dir_info.keys())   # list of key
    cnt = 1
    while True:
        search_dir_path = os.path.join(base_path, search_dir)
        print(search_dir_path)
        for _, _, files in os.walk(search_dir_path):
            for file in files:
                yield os.path.join(search_dir_path, file)
        search_dir = next(dir_iter)
        cnt += 1
        if search_dir == dir_list[-1] and cnt == len(dir_list):
            # when nothing found
            return

In [6]:
# 
def get_files(base_path, search_dir):
    while True:
        search_dir_path = os.path.join(base_path, search_dir)
        print(search_dir_path)
        for _, _, files in os.walk(search_dir_path):
            for file in files:
                yield os.path.join(search_dir_path, file)
        else:
            return
        
# return each article text
def format_text(data):
    head = head_reg.search(data)  # (<doc id=...>)
    id_tag = id_reg.search(head.group())
    doc_id = re.search(r'\d+', id_tag.group())  # extract id from header
    title_tag = title_reg.search(head.group())
    doc_title = re.search(r'".+"', title_tag.group())  
    doc_id = int(doc_id.group())
    title = doc_title.group().replace('"', '')      # delete ""
    # deete each header(<doc id=...>)and tag
    # extract only texts (delete title as some articles have duplicated tittles)
    text = data.replace(head.group(), '').rstrip('</doc>')[len(title):]
    return doc_id, title, text

# return lists of articles
def get_articlelist(f_path):
    f = open(f_path, 'r', encoding='utf-8')
    file_data = f.read()
    file_data = file_data.replace('\n', '')
    return doc_reg.findall(file_data)

In [7]:
def output_file(f_name, text):
    # delete special characters which can't be used for file names
    f_name = re.sub(r'¥|/|:|\*|\?|"|<|>|\||', '', f_name)
    path = OUTPUT_PATH + f_name
    f = open(path, 'w', encoding='utf-8')
    f.write(text)
    f.close()
    
def output_file(f_name, text):
    # delete special characters which can't be used for file names
    f_name = re.sub(r'¥|/|:|\*|\?|"|<|>|\||', '', f_name)
    path = OUTPUT_PATH + f_name
    f = open(path, 'w', encoding='utf-8')
    f.write(text)
    f.close()
    
if __name__ == '__main__':
    # 
    id_reg = re.compile(r'id="\d+"')
    title_reg = re.compile(r'title=".+"')
    doc_reg = re.compile(r'<doc id=.+?</doc>')
    head_reg = re.compile(r'<doc id.+?">')

In [8]:
# NEED TO CHANGE for differendt data
BASE_PATH = 'F:/Wikipedia/wikiextractor-master/wikiextractor'
CSV_FILE = BASE_PATH + '/princess.csv' # downloaded data from PetScan
WIKI_DIR = BASE_PATH + '/text'
OUTPUT_PATH = 'F:/Wikipedia/wikiextractor-master/wikiextractor/princess/'

In [9]:
# make dictionary of the max ID value in each folder
dir_info = OrderedDict()
dir_info = mapping_dir(WIKI_DIR)
counter = 1     # loop counter

In [10]:
# read csv
f = open(CSV_FILE, 'r', encoding='utf-8')
reader = csv.reader(f)
next(csv.reader(f))

['number', 'title', 'pageid', 'namespace', 'length', 'touched']

In [11]:
not_found = {}      # dict for not-found data

In [12]:
# measure time
start_time = time.time()

In [13]:
for line in reader:
        flag = False
        page_name = line[1]
        page_id = int(line[2])
        namespace = line[3]
        if namespace == "Category" or namespace == "Template":
            continue
        search_dir = get_search_dir(page_id, dir_info)
        for f_path in get_files(WIKI_DIR, search_dir):
            for d in get_articlelist(f_path):
                doc_id, title, text = format_text(d)
                if page_id == doc_id:
                    print(counter, title)
                    output_file(title, text)
                    counter += 1
                    flag = True
                    break
            if flag:
                break
        else:
            not_found[page_id] = page_name
            print("ページID: {}は見つかりませんでした.".format(page_id))
 
        end_time = time.time()
        processing_time = end_time - start_time
        print('processing_time(sec): ', processing_time)
 
        # print "not found" page and its ID
        for i, (_id, _title) in enumerate(not_found.items()):
            print(i, _id, _title)

F:/Wikipedia/wikiextractor-master/wikiextractor/text\AB
1 メアリー1世 (イングランド女王)
processing_time(sec):  1.1203010082244873
F:/Wikipedia/wikiextractor-master/wikiextractor/text\AD
2 ヴィクトリア (ドイツ皇后)
processing_time(sec):  1.774097204208374
F:/Wikipedia/wikiextractor-master/wikiextractor/text\AD
3 アリス (ヘッセン大公妃)
processing_time(sec):  3.1024136543273926
F:/Wikipedia/wikiextractor-master/wikiextractor/text\AD
4 ベアトリス (イギリス王女)
processing_time(sec):  4.883073091506958
F:/Wikipedia/wikiextractor-master/wikiextractor/text\AD
5 ダイアナ (プリンセス・オブ・ウェールズ)
processing_time(sec):  6.3051111698150635
F:/Wikipedia/wikiextractor-master/wikiextractor/text\AD
6 キャサリン・オブ・アラゴン
processing_time(sec):  9.26112961769104
F:/Wikipedia/wikiextractor-master/wikiextractor/text\AE
7 マデレーン (スウェーデン王女)
processing_time(sec):  11.560242176055908
F:/Wikipedia/wikiextractor-master/wikiextractor/text\AE
8 ヴィクトリア (スウェーデン皇太子)
processing_time(sec):  14.101186275482178
F:/Wikipedia/wikiextractor-master/wikiextractor/text\AF
9 アンナ・パヴロヴナ
pr

71 ジョゼフィーヌ・ド・ボアルネ (スウェーデン王妃)
processing_time(sec):  134.3012924194336
F:/Wikipedia/wikiextractor-master/wikiextractor/text\AN
72 マリア・ルイーザ・ディ・サヴォイア
processing_time(sec):  135.4089081287384
F:/Wikipedia/wikiextractor-master/wikiextractor/text\AN
73 ソフィア・ヴィルヘルミナ・アヴ・スヴェーリエ
processing_time(sec):  136.60622596740723
F:/Wikipedia/wikiextractor-master/wikiextractor/text\AN
74 ヴィクトリア・フォン・バーデン
processing_time(sec):  137.77514815330505
F:/Wikipedia/wikiextractor-master/wikiextractor/text\AN
75 オーガスタ・オブ・ウェールズ
processing_time(sec):  138.88273215293884
F:/Wikipedia/wikiextractor-master/wikiextractor/text\AN
76 ルイーズ・オブ・グレート・ブリテン
processing_time(sec):  140.29674363136292
F:/Wikipedia/wikiextractor-master/wikiextractor/text\AN
77 カタリーナ・フォン・ヴュルテンベルク
processing_time(sec):  142.24620366096497
F:/Wikipedia/wikiextractor-master/wikiextractor/text\AN
78 メアリー・オブ・グレート・ブリテン
processing_time(sec):  144.02327466011047
F:/Wikipedia/wikiextractor-master/wikiextractor/text\AN
79 シャーロット (ヴュルテンベルク王妃)
processing_time(se

139 アーデルグンデ・フォン・バイエルン (1823-1914)
processing_time(sec):  266.0196087360382
F:/Wikipedia/wikiextractor-master/wikiextractor/text\AN
140 ヘレナ (イギリス王女)
processing_time(sec):  268.3844795227051
F:/Wikipedia/wikiextractor-master/wikiextractor/text\AN
141 ヒルデガルト・フォン・バイエルン (1825-1864)
processing_time(sec):  271.0876307487488
F:/Wikipedia/wikiextractor-master/wikiextractor/text\AN
142 ルイーズ (アーガイル公爵夫人)
processing_time(sec):  273.8641724586487
F:/Wikipedia/wikiextractor-master/wikiextractor/text\AN
143 ルイーズ (ファイフ公爵夫人)
processing_time(sec):  277.34509563446045
F:/Wikipedia/wikiextractor-master/wikiextractor/text\AN
144 メアリー (ハーウッド伯爵夫人)
processing_time(sec):  282.4490485191345
F:/Wikipedia/wikiextractor-master/wikiextractor/text\AN
145 カタリナ・デ・ランカステル
processing_time(sec):  286.5400812625885
F:/Wikipedia/wikiextractor-master/wikiextractor/text\AN
146 ジビラ・フォン・ザクセン＝コーブルク＝ゴータ
processing_time(sec):  290.6611843109131
F:/Wikipedia/wikiextractor-master/wikiextractor/text\AN
147 マルゲリータ・ディ・サヴォイア (マントヴァ公妃)
pr

206 マリー＝クリスティーヌ (マイケル王子夫人)
processing_time(sec):  397.60141158103943
F:/Wikipedia/wikiextractor-master/wikiextractor/text\AR
207 アレクサンドラ (レディ・オギルヴィ)
processing_time(sec):  398.71985626220703
F:/Wikipedia/wikiextractor-master/wikiextractor/text\AR
208 アレクサンドリーネ・フォン・プロイセン (1842-1906)
processing_time(sec):  399.92845249176025
F:/Wikipedia/wikiextractor-master/wikiextractor/text\AR
209 パウリーネ・フォン・ヴュルテンベルク (1810-1856)
processing_time(sec):  401.6835296154022
F:/Wikipedia/wikiextractor-master/wikiextractor/text\AR
210 マドレーヌ・ド・フランス (ビアナ公妃)
processing_time(sec):  403.5812666416168
F:/Wikipedia/wikiextractor-master/wikiextractor/text\AR
211 ルイーズ・ド・サヴォワ
processing_time(sec):  405.3673985004425
F:/Wikipedia/wikiextractor-master/wikiextractor/text\AR
212 アレクサンドラ (第2代ファイフ公爵)
processing_time(sec):  407.1526424884796
F:/Wikipedia/wikiextractor-master/wikiextractor/text\AR
213 フリーデリケ・フォン・ハノーファー (1848-1926)
processing_time(sec):  409.4876687526703
F:/Wikipedia/wikiextractor-master/wikiextractor/text\AR


273 ゾフィー・フォン・ザクセン (1845-1867)
processing_time(sec):  523.9069511890411
F:/Wikipedia/wikiextractor-master/wikiextractor/text\AV
274 マリア・フォン・バイエルン
processing_time(sec):  524.2936503887177
F:/Wikipedia/wikiextractor-master/wikiextractor/text\AV
275 マリーア・イザベッラ・ダズブルゴ＝トスカーナ
processing_time(sec):  524.6113867759705
F:/Wikipedia/wikiextractor-master/wikiextractor/text\AV
276 ステファニー・ド・ベルジック
processing_time(sec):  525.034907579422
F:/Wikipedia/wikiextractor-master/wikiextractor/text\AV
277 ルイーズ・ド・ベルジック
processing_time(sec):  525.4800138473511
F:/Wikipedia/wikiextractor-master/wikiextractor/text\AV
278 クレマンティーヌ・ド・ベルジック
processing_time(sec):  525.8675820827484
F:/Wikipedia/wikiextractor-master/wikiextractor/text\AV
279 アンリエット・ド・ベルジック
processing_time(sec):  526.259425163269
F:/Wikipedia/wikiextractor-master/wikiextractor/text\AV
280 ジョゼフィーヌ・ド・ベルジック
processing_time(sec):  526.6833713054657
F:/Wikipedia/wikiextractor-master/wikiextractor/text\AV
281 マルガレーテ・ゾフィー・フォン・エスターライヒ
processing_time(sec):  527.

340 アミーリア・オブ・ザ・ユナイテッド・キングダム
processing_time(sec):  612.2522096633911
F:/Wikipedia/wikiextractor-master/wikiextractor/text\AW
341 ソフィア・オブ・グロスター
processing_time(sec):  614.6174101829529
F:/Wikipedia/wikiextractor-master/wikiextractor/text\AW
342 カタリーナ・フォン・ヴュルテンベルク (1821-1898)
processing_time(sec):  616.7697749137878
F:/Wikipedia/wikiextractor-master/wikiextractor/text\AW
343 イザベラ・フォン・エスターライヒ (1888-1973)
processing_time(sec):  619.2373151779175
F:/Wikipedia/wikiextractor-master/wikiextractor/text\AW
344 マリー＝アデライード・ド・リュクサンブール
processing_time(sec):  621.7526488304138
F:/Wikipedia/wikiextractor-master/wikiextractor/text\AW
345 マリー＝ガブリエル・ド・リュクサンブール
processing_time(sec):  624.2306997776031
F:/Wikipedia/wikiextractor-master/wikiextractor/text\AX
346 マティルデ・フォン・ザクセン
processing_time(sec):  624.2886819839478
F:/Wikipedia/wikiextractor-master/wikiextractor/text\AX
347 エステル (エステルイェートランド公)
processing_time(sec):  624.5041193962097
F:/Wikipedia/wikiextractor-master/wikiextractor/text\AX
348 マリー・フォン・ヴュルテ

408 マリーア・ヴィットーリア・フランチェスカ・ディ・サヴォイア
processing_time(sec):  710.5819027423859
F:/Wikipedia/wikiextractor-master/wikiextractor/text\BF
409 アリックス・フォン・ザクセン
processing_time(sec):  712.3857743740082
F:/Wikipedia/wikiextractor-master/wikiextractor/text\BF
410 マリーア・カテリーナ・ブリニョーレ・サーレ
processing_time(sec):  714.1754329204559
F:/Wikipedia/wikiextractor-master/wikiextractor/text\BF
411 マリー・テレーズ・ド・ショワズール
processing_time(sec):  716.234542131424
F:/Wikipedia/wikiextractor-master/wikiextractor/text\BF
412 マリー・ド・ロレーヌ (1674-1724)
processing_time(sec):  718.1067395210266
F:/Wikipedia/wikiextractor-master/wikiextractor/text\BF
413 ルイーズ・ドーモン
processing_time(sec):  719.9536669254303
F:/Wikipedia/wikiextractor-master/wikiextractor/text\BF
414 カロリーヌ・ジベール
processing_time(sec):  721.970507144928
F:/Wikipedia/wikiextractor-master/wikiextractor/text\BF
415 フロレスティーヌ・ド・モナコ
processing_time(sec):  724.5199835300446
F:/Wikipedia/wikiextractor-master/wikiextractor/text\BF
416 シャルロット・ド・モナコ
processing_time(sec):  726.454940